In [ ]:
import os
import gdal
import numpy as np
import matplotlib.pyplot as plt
from datetime import date
import skimage.transform as st 
%matplotlib inline

In [ ]:
import PIL
from PIL import Image
from matplotlib import cm

In [ ]:
import cv2
import time
import zipfile
import datetime

In [ ]:
from pathlib import Path

In [ ]:
#from rasterio.plot import show
#from rasterio.merge import merge
#import rasterio as rio

In [ ]:
#pip install xarray-spatial

In [ ]:
#pip install rasterio

In [ ]:
pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 14.8 MB/s 
     |████████████████████████████████| 16.7 MB 57.1 MB/s 
     |████████████████████████████████| 6.3 MB 50.0 MB/s 


In [ ]:
pip install asf_search

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 51 kB 304 kB/s 
     |████████████████████████████████| 288 kB 23.4 MB/s 
     |████████████████████████████████| 749 kB 34.7 MB/s 
  Attempting uninstall: regex
    Found existing installation: regex 2022.6.2
    Uninstalling regex-2022.6.2:
      Successfully uninstalled regex-2022.6.2


In [ ]:
pip install mapclassify

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon
import os

In [ ]:
#pip install rasterio

In [ ]:
import asf_search as asf

In [ ]:
basepath = !pwd

In [ ]:
basepath = basepath[0]

Setting Up working directories for Sentinel 1 data download and processing

In [ ]:
download_dir = 'downloads'
geotrasform_dir = 'geotrasform'
s1cuts = 's1cuts2'
mosaic = 'mosaic'
unziped = 'unziped'
if not os.path.exists(download_dir):
    os.makedirs(download_dir)

if not os.path.exists(s1cuts):
    os.makedirs(s1cuts)

if not os.path.exists(geotrasform_dir):
    os.makedirs(geotrasform_dir)

if not os.path.exists(mosaic):
    os.makedirs(mosaic)

if not os.path.exists(unziped):
    os.makedirs(unziped)

In [ ]:
wkt_aoi = 'POLYGON((44.2512 -10.1035,40.4229 -10.3482,40.9201 -16.2483,35.103 -20.3073,35.6996 -24.6248,33.2137 -25.1209,33.2634 -26.9967,39.6274 -26.6862,44.2512 -10.1035))'

In [ ]:
username = 'Dercio15'
password = '********'

try:
    user_pass_session = asf.ASFSession().auth_with_creds(username, password)
except asf.ASFAuthenticationError as e:
    print(f'Auth failed: {e}')
else:
    print('Success!')

Success!


In [ ]:
import datetime

In [ ]:
starting_date = datetime.date.today() - datetime.timedelta(days=12)
ending_date = datetime.date.today()

Download Sentinel 1 data

In [ ]:
def downloadS1GDR(download_dir, starting_date, ending_date):
  results = asf.geo_search(
    intersectsWith=wkt_aoi,
    platform= asf.PLATFORM.SENTINEL1,
    processingLevel=[asf.PRODUCT_TYPE.GRD_HD],
    #flightDirection=asf.FLIGHT_DIRECTION.ASCENDING,
    beamMode=asf.BEAMMODE.IW,
    start = starting_date,
    end = ending_date)
  print(f'Total Images Found: {len(results)}')
  ### Save Metadata to a Dictionary
  metadata = results.geojson()
  print("Starting download")
  results.download(path=download_dir, session=user_pass_session)
  print("Ending download")
  outfiles = []
  for path in Path(download_dir).rglob('*.zip'):
      outfiles.append(str(path.resolve()))
  return outfiles, metadata

In [ ]:
s1gdr, metadata = downloadS1GDR(download_dir, starting_date, ending_date)

Total Images Found: 24
Starting download
Ending download


In [ ]:
s1gdr

Processing

In [ ]:
def unzips1(files, unziped):
  S1_GDR = []
  for file in files:
    with zipfile.ZipFile(file, 'r') as zip_ref:
      zip_ref.extractall(unziped)
    print(f'{file} was uniziped with sucess')

  for path in Path(unziped).rglob('*.tiff'):
    if path.name[11:13] == 'vv':
      S1_GDR.append(str(path.resolve()))
  return S1_GDR

In [ ]:
unipeds1 = unzips1(s1gdr, unziped)

In [ ]:
unipeds1

In [ ]:
import shutil

In [ ]:
shutil.rmtree(download_dir, ignore_errors=False, onerror=None)

Convert from Radar Coordinates to UTM

In [ ]:
def geotransform(s1vvlist,outlocation, xres=10, yres=10, espg="EPSG:2736"):
  outlocation = outlocation+'/'
  outlistgeo = []
  for image in s1vvlist:
    scene = gdal.Open(image)
    Outname = image.split(".t")[0][-33:-18]
    Outname = f"{outlocation}S1_IW_VV_{Outname}.tif"
    wrp =gdal.Warp(Outname, scene, xRes = xres, yRes = yres, dstNodata =0, dstSRS = espg, outputType=gdal.GDT_UInt16)
    scene = None
    wrp = None
    time.sleep(5)
    print(f"{Outname} geotrasformed")
  
  for path in Path(outlocation).rglob('*.tif'):
      outlistgeo.append(str(path.resolve()))
  return outlistgeo

In [ ]:
#geocode image
outscene = geotransform(unipeds1, geotrasform_dir)

In [ ]:
outscene

In [ ]:
shutil.rmtree(unziped, ignore_errors=False, onerror=None)

### Cut the raster in 30 iqual pieaces

In [ ]:
def cutimage(scenes, outlocation, num_cuts = 30):
  cuttedlist = []

  for outscene in scenes:
    gdataset = gdal.Open(outscene)
    dimensions = gdataset.GetGeoTransform()
    #Get resolution info
    xmin = dimensions[0]
    ymax = dimensions[3]
    res = dimensions[1]

    #len of raster
    xlen = res*gdataset.RasterXSize
    ylen = res*gdataset.RasterYSize

    #image lenth
    xsize = round(xlen/num_cuts)
    ysize = round(ylen/num_cuts)

    #x and y coordinates we want to create
    xsteps = [xmin + xsize * i for i in range(num_cuts+1)]
    ysteps = [ymax - ysize * i for i in range(num_cuts+1)]

    for i in range(num_cuts):
      for j in range(num_cuts):
          xmin = xsteps[i]
          xmax = xsteps[i+1]
          ymax = ysteps[j]
          ymin = ysteps[j+1]
          outscene = outscene.split('.')[0].split('/')[-1]
          outname = f"{outlocation}/{outscene}"+str(i)+"a"+str(j)+"b"+".tif"
          # use gdal warp
          gdal.Warp(outname, gdataset, outputBounds = (xmin, ymin, xmax, ymax), dstNodata = 0, outputType=gdal.GDT_UInt16)
          print(f"{outname} cutted with success")
  for path in Path(outlocation).rglob('*.tif'):
    if path.name[0:2] == 'S1':
      cuttedlist.append(str(path.resolve()))
  return cuttedlist

In [ ]:
cuts1 = cutimage(outscene, s1cuts)

In [ ]:
len(cuts1)

19800

### Processing each raster, make inference and put it in original shape and format

In [ ]:
model_path = '/content/drive/MyDrive/Colab Notebooks/sarshipv30.pt'

In [ ]:
!pip install -q --upgrade selectivesearch torch_snippets

     |████████████████████████████████| 62 kB 676 kB/s 
     |████████████████████████████████| 58 kB 5.8 MB/s 
     |████████████████████████████████| 78 kB 7.8 MB/s 
     |████████████████████████████████| 237 kB 45.7 MB/s 
     |████████████████████████████████| 1.6 MB 50.3 MB/s 
     |████████████████████████████████| 175 kB 60.2 MB/s 
     |████████████████████████████████| 2.2 MB 29.5 MB/s 
     |████████████████████████████████| 51 kB 7.0 MB/s 


In [ ]:
from torch_snippets import *
from PIL import Image
from torchvision import transforms, models, datasets
from torch_snippets import Report
from torchvision.ops import nms
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

device = 'cuda' if torch.cuda.is_available() else 'cpu'

def get_model():
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, 2)
    return model

In [ ]:
def loadmodel(model_path, device=device):
  model = get_model().to(device)
  model.load_state_dict(torch.load(model_path, map_location=torch.device(device)))
  return model

In [ ]:
model = loadmodel(model_path)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [ ]:
def prepereimage(img_path):
  img = Image.open(img_path).convert('RGB')
  img = img.resize((800, 800), Image.BILINEAR)
  img = np.asarray(img)
  img = img/255.
  img = torch.tensor(img).to(device).float()
  img = img.permute(2, 0, 1)
  img = img[None, :]
  return img

In [ ]:
img_path = cuts1[0]

In [ ]:
imgTensor = prepereimage(img_path)

In [ ]:
target2label ={'ship': 1, 'background': 0}

In [ ]:
len(target2label)

2

In [ ]:
def predict(imgTensor, model):
  model.eval()
  outputs = model(imgTensor)
  return outputs

In [ ]:
outputs = predict(imgTensor, model)

In [ ]:
from torchvision.ops import nms
def decode_output(output, proba_limit=0.9):
    'convert tensors to numpy arrays'
    bbs = output['boxes'].cpu().detach().numpy().astype(np.uint16)
    labels = np.array(output['labels'].cpu().detach().numpy())
    confs = output['scores'].cpu().detach().numpy()
    #confs = np.where(confs>0.8)
    ixs = nms(torch.tensor(bbs.astype(np.float32)), torch.tensor(confs), 0.1)
    bbs, confs, labels = [tensor[ixs] for tensor in [bbs, confs, labels]]

    if len(ixs) == 1:
        bbs, confs, labels = [np.array([tensor]) for tensor in [bbs, confs, labels]]
    bbs, confs, labels=bbs.tolist(), confs.tolist(), labels.tolist()
    indices = []
    for i in range(len(confs)):
      if confs[i]>proba_limit:
        indices.append(i)
    confs = [confs[i] for i in indices]
    bbs = [bbs[i] for i in indices]
    labels = [labels[i] for i in indices]
    return bbs, confs, labels

In [ ]:
for ix, output in enumerate(outputs):
    bbs, confs, labels = decode_output(output)
    info = [f'{l}@{c:.2f}' for l,c in zip(labels, confs)]
    img = imgTensor[ix][1]*255
    img = img.cpu().detach().numpy().astype(np.uint16)
    #show(img.cpu(), bbs=bbs, sz=5)
    print(bbs)
    for i in bbs:
      xmin, ymin, w, h = i
      widh = int(w)
      heigh = int(h)
      X = int(xmin)
      Y = int(ymin)
      showim = cv2.rectangle(img, (X, Y), (widh, heigh), (0, 0, 255), 2)
    #res, im_png = cv2.imencode(".png", showim)
    plt.figure(figsize=(15, 20))
    plt.imshow(showim, cmap='gray')

In [ ]:
def rescalebbs(bbs, img, originalsize=800):
  bbs_res = []
  h, w = img.shape
  for i in bbs:
      xmin, ymin, xmax, ymax = i
      x_scale =  w/originalsize
      y_scale =  h/originalsize

      xmin = int(np.round(xmin*x_scale))
      ymin =  int(np.round(ymin*y_scale))
      xmax =  int(np.round(xmax*x_scale))
      ymax =  int(np.round(ymax*y_scale))
      bbs_res.append([xmin, ymin, xmax, ymax])
  return bbs_res

In [ ]:
def bboxinpixel(bbs:list, GT):
  utmcor = []
  for i in bbs:
    xmin, ymin = gdal.ApplyGeoTransform(GT, i[0], i[1])
    xmax, ymax = gdal.ApplyGeoTransform(GT, i[2], i[3])
    utmcor.append([xmin, ymin, xmax, ymax])
  return utmcor

In [ ]:
# function to return polygon
def utm_polygon(utmcor:list):
  ships, polygons = [], []
  for ix, cord in enumerate(utmcor):
    ships.append('ship'+str(ix))
    corner1 = [cord[0], cord[1]]
    corner2 = [cord[2], cord[1]]
    corner3 = [cord[2], cord[3]]
    corner4 = [cord[0], cord[3]]
    polygons.append(Polygon([corner1, corner2, corner3, corner4]))
  #ds = pd.DataFrame({'ShipID': ships})
  #dsgeo = gpd.GeoDataFrame(crs='epsg:2736', geometry=polygons)
  return ships, polygons

In [ ]:
import folium
import mapclassify

In [ ]:
def PreditShip(cuttedlist):

  ships_list, polygons_list, confs_list = [], [], []

  model = loadmodel(model_path)
  for images in cuttedlist:
    ds = gdal.Open(images)
    im = ds.GetRasterBand(1).ReadAsArray()
    gt = ds.GetGeoTransform()
    proj = ds.GetProjection()

    #Prepare Image
    img = prepereimage(images)

    #Make predition
    output = predict(img, model)

    #process the outputs
    for ix, output in enumerate(output):
      try:
        bbs, confs, labels = decode_output(output)

        #Rescale bbs for original image
        bbs_res = rescalebbs(bbs, im)

        #convert pixel coordinates to utm coordinates
        utmcoor = bboxinpixel(bbs_res, gt)

        #generate coordinates 
        ships, polygons = utm_polygon(utmcoor)

        #save values to a list
        ships_list.append(ships)
        polygons_list.append(polygons)
        confs_list.append(confs)
      except:
        print('couldnt process')
  return ships_list, polygons_list, confs_list

In [ ]:
ships_list, polygons_list, confs_list = PreditShip(cuts1)

In [ ]:
def processpolygons(polygons_list, confs_list):
  poli = []
  for i in polygons_list:
    for j in i:
      poli.append(j)
  dsgeo = gpd.GeoDataFrame(crs='epsg:2736', geometry=poli)

  confs = []
  for i in confs_list:
    for j in i:
      confs.append(j)
  dsgeo['probability'] = confs
  return dsgeo

In [ ]:
dsgeo = processpolygons(polygons_list, confs_list)

In [ ]:
dsgeo.head()

In [ ]:
dsgeo.shape

In [ ]:
df = dsgeo.to_crs(epsg=4326)
print(df.crs)
df.head()

In [ ]:
m = folium.Map(location=[-26.8, 32.4], zoom_start=10, tiles='CartoDB positron')
folium.GeoJson(df).add_to(m)
folium.LatLngPopup().add_to(m)
m

## Mosaic the cutted Raster to it original form

In [ ]:
import glob

In [ ]:
l = os.listdir('geotrasform')

In [ ]:
sc =[]
for i in l:
  n = 'geotrasform/'+i
  sc.append(n)

In [ ]:
ds = gdal.Open(sc[3])
arr = ds.GetRasterBand(1).ReadAsArray()

In [ ]:
vmin = np.percentile(arr.flatten(), 5)
vmax = np.percentile(arr.flatten(), 95)

In [ ]:
plt.figure(figsize=(14, 12), dpi=80)
plt.imshow(arr,vmin=vmin, vmax=vmax, cmap='gray')
#plt.colorbar()
plt.show()

In [ ]:
def mergescenes(scenelist,outname, xres=10, yres=10):
  vrt_name=f"{outname}/S1_IW_VV_{str(datetime.datetime.now())}.vrt"
  vrt = gdal.BuildVRT(vrt_name, scenelist)
  outname = f"{outname}/S1_IW_VV_Merge{str(datetime.datetime.now())}.tif"
  gdal.Translate(outname, vrt, xRes = xres, yRes = yres, noData=0, outputType=gdal.GDT_UInt16)
  vrt = None
  time.sleep(10)
  return outname

In [ ]:
name = mergescenes(sc, 'mosaic')

In [ ]:
name = '/content/mosaic/S1_IW_VV_Merge2022-10-04 12:03:17.765251.tif'

In [ ]:
ds_1 = gdal.Open(name)
array_ = ds_1.GetRasterBand(1).ReadAsArray()
gt_1 = ds_1.GetGeoTransform()
proj_1 = ds_1.GetProjection()

In [ ]:
array_.shape

In [ ]:
array_2 = cv2.resize(array_, dsize=(4277, 5294), interpolation=cv2.INTER_CUBIC)

In [ ]:
vmin = np.percentile(array_2.flatten(), 5)
vmax = np.percentile(array_2.flatten(), 95)

In [ ]:
plt.figure(figsize=(15, 20))
plt.imshow(array_2,vmin=vmin, vmax=vmax, cmap='gray')
#plt.colorbar()
plt.show()

# Generate Tiles

In [ ]:
pip install gdal2tiles

In [ ]:
!gdal2tiles.py -help

In [ ]:
import gdal2tiles

In [ ]:
source_path = "Enter Source Path here"
destination_path = "Enter Source Path here"

In [ ]:
def create_tiles(source_path, destination_path):
  """
  Method to generate tiles at different zoom levels using gdal2tiles library.
  Args:
  source_path : path for the source file which needs to be tiled
  destination_path : path for the output directory where the tiles will be generated
  """
  options = {'zoom': 16,
        'nb_processes': 4,
        'tile_size': 256,
        'srs':'EPSG:4326'
        }
  gdal2tiles.generate_tiles(source_path, destination_path, **options)

In [ ]:
create_tiles(source_path, destination_path)